# <span style="color:#336699">3º BIG TechTalks - Acesso, Visualização e Processamento de Imagens Sentinel-2 utilizando Python - Usando o serviço Tile Map Service (TMS) para visualização de imagens</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Gilberto Ribeiro de Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Karine Reis Ferreira<sup><a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Marcos Adami<sup><a href="https://orcid.org/0000-0003-4247-4477"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Thales Sehn Körting<sup><a href="https://orcid.org/0000-0002-0876-0501"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Rennan de Freitas Bezerra Marujo<sup><a href="https://orcid.org/0000-0002-0082-9498"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Divisão de Observação da Terra e Geoinformática, Instituto Nacional de Pesquisas Espaciais (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contato: <a href="rennan.marujo@inpe.br/">rennan.marujo@inpe.br</a>
    <br/><br/>
    Última Atualização: 24 de Abril de 2025
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
    <b>Resumo.</b> Este Jupyter Notebook é parte do 3º BIG TechTalks - Acesso, Visualização e Processamento de Imagens Sentinel-2 utilizando Python. Na <em>Plataforma Brazil Data Cube</em> as coleções de imagens e cubos de dados podem ser consultadas e visualizadas através de uma interface gráfica com o usuário denominada <a href="https://data.inpe.br/bdc/explorer/explore" target="_blank">Data Cube Explorer</a> ou através de interfaces de programação de aplicações (API) baseadas em diversos serviços web, como o <a href="https://stacspec.org/" target="_blank">SpatioTemporal Asset Catalog (STAC)</a> e <a href="https://wiki.osgeo.org/wiki/Tile_Map_Service_Specification" target="_blank">Tile Map Service (TMS)</a>, entre outros. Este Jupyter Notebook apresenta uma visão geral do uso integrado do serviços TMS para visualização dos produtos de dados de sensoriamento remoto com base nas tecnologias da <em>Plataforma Brazil Data Cube</em>.
</div>

# Preparação do Ambiente
<hr style="border:1px solid #0077b9;">

Assim como fizemos no Jupyter Notebook do STAC vamos carregar um shapefile e alguns items da coleção Sentinel-2.

Caso não tenha o Client STAC instalado em seu ambiente execute:

In [ ]:
# !pip install pystac-client

Para visualizarmos o polígono da gleba sobre um mapa de referência, é possível utilizar as bibliotecas `ipyleaflet` e `ipywidgets`.

In [ ]:
# Área de exibição de camadas (mapa) e mapas base (basemaps)
from ipyleaflet import Map, basemaps, basemap_to_tiles

# Controles sobre o mapa
from ipyleaflet import FullScreenControl, LayersControl, ScaleControl, SplitMapControl, WidgetControl

# Tipos de Camadas
from ipyleaflet import GeoJSON, TileLayer

# Layout
from ipywidgets import IntSlider, Layout

Vamos ler um shapefile e mostra-lo num mapa leaflet:

In [ ]:
import geopandas as gpd
import io
import os
import requests
import tempfile
import warnings
import zipfile

from ipyleaflet import GeoData

zipfile_url = "https://github.com/brazil-data-cube/code-gallery/raw/master/jupyter/Data/2025-sbsr/LEM_dataset_small.zip"
response = requests.get(zipfile_url)
with tempfile.TemporaryDirectory() as tmpdir:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(tmpdir)

        shp_file = [f for f in os.listdir(tmpdir) if f.endswith('.shp')][0]
        shp_path = os.path.join(tmpdir, shp_file)

        my_shp = gpd.read_file(shp_path)

        warnings.filterwarnings("ignore")
        geometry_union = my_shp.geometry.union_all()
        bbox = geometry_union.bounds
        centroide = geometry_union.centroid

centro = [my_shp.geometry.centroid.y.mean(), my_shp.geometry.centroid.x.mean()]
warnings.filterwarnings("default")

m = Map(center=centro, zoom=11)

geodata = GeoData(
    geo_dataframe=my_shp,
    style={
        'color': 'blue',
        'fillColor': 'green',
        'opacity': 0.8,
        'weight': 1.5,
        'fillOpacity': 0.2
    },
    hover_style={'fillColor': 'red', 'fillOpacity': 0.4},
    name="Shapefile"
)
m.add(geodata)

m

Agora vamos buscar alguns `items` utilizando STAC:

In [ ]:
import pystac_client

catalogo = pystac_client.Client.open('https://data.inpe.br/bdc/stac/v1/')
sentinel2 = catalogo.get_collection("S2_L2A-1")

item_search = catalogo.search(
    collections=[sentinel2],
    bbox=bbox,
    datetime='2024-01-01/2025-03-21',
    limit = 100
)

items_list = list(item_search.items())

items = item_search.item_collection()

item = items[0]

# Usando o ipyleaflet para visualização de imagens a partir de um serviço TMS
<hr style="border:1px solid #0077b9;">

Vamos trabalhar específicamente com um poligono, no caso o de índice `22` (vigésimo terceiro poligono do shapefile). Podemos criar uma camada para armazenar as informações desse poligono:

In [ ]:
import shapely

polygon = my_shp.iloc[22]
geometry = polygon.geometry
centroide = geometry.centroid

camada_poly = GeoJSON(
    name="Polígono da Gleba",
    data=shapely.geometry.mapping(geometry),
    style={ 'color': 'SteelBlue', 'opacity': 1, 'fillOpacity': 0.1, 'weight': 5 },
    hover_style={ 'color': 'IndianRed', 'opacity': 1, 'fillOpacity': 0.1, 'weight': 5 }
)

Vamos visualiza-lo:

In [ ]:
mapa = Map(zoom=15,
           scroll_wheel_zoom=True,
           center=(centroide.y, centroide.x),  # [lat, lon]
           layout=Layout(width='80%', height='500px')
           )

mapa.add(camada_poly)

mapa

Podemos extrair o bounding box desse poligono com:

In [ ]:
bbox_poly = geometry.bounds

Neste exemplo, usaremos a imagem em cores verdadeiras disponível no endereço fornecido pelo primeiro *item* do resultado da pesquisa anterior.

In [ ]:
assets = items[0].assets
assets

In [ ]:
tci_asset = assets['TCI']
tci_asset.href

Vamos criar uma camada para essa cena:

In [ ]:
camada_cena = TileLayer(
    name=items[0].id,
    url="https://data.inpe.br/bdc/tms/tiles/WebMercatorQuad/{z}/{x}/{y}" + f"?url={tci_asset.href}"
)

Podemos visualiza-la sobre o mapa:

In [ ]:
mapa_tms = Map(zoom=12,
               scroll_wheel_zoom=True,
               layout=Layout(width='80%', height='500px'),
               center=(centroide.y, centroide.x),  # [lat, lon]
               )

mapa_tms.add(camada_poly)
mapa_tms.add(camada_cena)

mapa_tms.add(LayersControl(position='topright'))
mapa_tms.add(FullScreenControl())
mapa_tms.add(ScaleControl(position='bottomleft'))

mapa_tms.fit_bounds([[bbox_poly[1], bbox_poly[0]], [bbox_poly[3], bbox_poly[2]]])

display(mapa_tms)

Agora, vamos gerar uma visualização que possibilite comparar duas imagens da mesma região em tempos distintos. As imagems que utilizaremos serão a primeira e a última na nossa lista de itens recuperados do STAC:

In [ ]:
items[20].id

In [ ]:
items[0].id

Repare no nome dos identificadores das cenas que essas imagens são do dia `17/02/2025` e `19/03/2025`, respectivamente.

In [ ]:
camada_cena_1 = TileLayer(
    name=items[20].id,
    url="https://data.inpe.br/bdc/tms/tiles/WebMercatorQuad/{z}/{x}/{y}" + f"?url={items[20].assets['TCI'].href}"
)

camada_cena_2 = TileLayer(
    name=items[0].id,
    url="https://data.inpe.br/bdc/tms/tiles/WebMercatorQuad/{z}/{x}/{y}" + f"?url={items[0].assets['TCI'].href}"
)


mapa_comp = Map(zoom=13, scroll_wheel_zoom=True, layout=Layout(width='80%', height='500px'), center=(centroide.y, centroide.x))

mapa_comp.add(camada_poly)

control = SplitMapControl(left_layer=camada_cena_1, right_layer=camada_cena_2)

mapa_comp.add(control)


mapa_comp.add(LayersControl(position='topright'))
mapa_comp.add(FullScreenControl())
mapa_comp.add(ScaleControl(position='bottomleft'))

mapa_comp.fit_bounds([[polygon.geometry.bounds[1], polygon.geometry.bounds[0]], [polygon.geometry.bounds[3], polygon.geometry.bounds[2]]])

display(mapa_comp)